https://github.com/rimchang/python_ml_ch9




# Embedding a Machine Learning Model into a Web Application

### 이번장의 목표

- Saving the current state of a trained machine learning model
    - 학습된 머신러닝 모델의 상태를 저장
- Using SQLite databases for data storage
    - DB로 SQLite 를 사용
- Developing a web application using the popular Flask web framework
    - Flask를 이용해 web app을 향상시킴
- Deploying a machine learning application to a public web server
    - 웹서버에 머신러닝 app을 deploy 함
    
# Serializing(직렬화) fitted scikit-learn estimators

머신러닝 모델을 트레이닝 하는 것은 계산적으로 매우 비싸다. 확실히 우리는 우리의 모델을 매번 트레이닝 시키는 것을 원하지 않는다. 하나의 옵션은 <strong>model persistence</strong> 인데 이는 파이썬의 pickle 모듈을 이용한다. 이것은 파이썬 object를 serialize , deserialize 하게 해준다. 그래서 우리는 우리의 classfier를 저장할 수 있고 우리가 새로운 sample이 필요할 때 다시 트레이닝 시키지 않아도 된다. 밑의 예제를 실행하기 전에 ch8 의 코드를 실행 시켜놔라

### serializing - python object 를 json같은 웹에서 사용가능한 형식으로 변환

### deserializing - json 같은 object를 python 에서 사용가능한 형식으로 변환(dict,list 등)

CH8 에서 로지스틱으로 sentiment 분석 했던 모델을 pickle 로 저장해놓고 로드하는 것으로 사용 할 것

In [ ]:
import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop,
        open(os.path.join(dest, 'stopwords.pkl'),'wb'),
        protocol=4)
pickle.dump(clf,
        open(os.path.join(dest, 'classifier.pkl'), 'wb'),
        protocol=4)

# stopwords 를 불러오고 그걸 통해 toknizer 

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle


cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir,
                'pkl_objects',
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

HasingVectorizer 는 scikit-learn 에서 text 처리의 부분 패키지이다. tfidfvectorize, Hasingvectorizer, countvectorizer 의 종류가 있으며 hasingvectorizer 는 각각 단어의 count를 해줘 vector로 만들어주는 함수인데 countvecotrizer 보다 빠르다

위의 코드를 통해 우리는 movieclassifier 디렉토리를 만들고 그안에 우리의 웹 어플리케이션을 위한 파일과 데이터를 저장 할 것이다. moviecalssifier 디렉토리 안에서 우리는 pkl_objects 하위 폴더를 만들고 python object를 직렬화하여 저장 할 것이다. pickle의 dump 메소드를 통해 우리는 ch8에서 만들었던 로지스틱 리그레션 모델과 Nltk 의 stop word 를 저장할 것이다. 그러면 우리의 서버에는 nltk를 설치할 필요가 없다. dump 메소드는 첫번째 인자로 우리가 원하는 pickle object를 받고 두번째 인자로 파이썬 object로 쓰여질 것을 넣는다. open function 안의 wb 인자는 우리가 파일을 binary mode 로 열거라는 것이도 protocol=4 는 파이썬 3.4 에서는 추가된 가장 효율적인 pcikle protocol 이다. 

우리는 HashingVectorizer 를 pickle 할 필요는 없는다. 그대신 우리는 파이썬 스크립트를 새로 만들어서 현재 python session 에 import 한다. 

# vectorize 와 calssfier 모델이 잘 돌아가는지 TEST

In [ ]:
import pickle
import re
import os
from vectorizer import vect
clf = pickle.load(open(os.path.join('pkl_objects','classifier.pkl'),'rb'))

import numpy as np
label = {0:'negative', 1:'positive'}
example = ['I love this movie']
X = vect.transform(example)
print(X)
print('Prediction: %s\nProbability: %.2f%%' %\
    (label[clf.predict(X)[0]],
    np.max(clf.predict_proba(X))*100))

# classfiy함수와 train 함수를 정의

이는 일반적인 파이썬 파일이며 이를 쟝고 프레임웤 안에 import 함으로써 사용 할 것이다

In [ ]:
# movieclassfier/movie_model.py

import os,pickle
from vectorizer import vect
import numpy as np
######## Preparing the Classifier
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir,'pkl_objects/classifier.pkl'), 'rb'))

def classify(document):
    label = {0: 'negative', 1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = np.max(clf.predict_proba(X))
    return label[y], proba
    
def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])

# django framework

django 는 MTV 패턴을 따른다

<img src="http://s.profissionaisti.com.br/wp-content/uploads/2009/03/django-framework-266x300.png" width=300 />

우리의 웹 서버 주소가 https://python-ml-ch9-rimchang.c9users.io/ 라고 하자

만약 https://python-ml-ch9-rimchang.c9users.io/<strong>home</strong> 이라는 request 를 사용자가 보내면

urlpatcher 에서  /home 에 맞는 view 에 연결 해준다. 

연결된 view에서 데이터를 처리해 주고 알맞은 template에 랜더링 해준후 유저에게 보내준다

# Setting up a SQLite database for data storage

입력받은 데이터와 그 결과값들을 저장 하기 위해 sqlite db에 저장할 수 있는 구조를 만들어 준다. 이때 결과값들을 따로 저장 하는 이유는 추후에 모델을 다시 트레인 시키기 위해서이다.

책에서는 파이썬 내장 sqlite3 를 가지고 db을 직접 만들었지만 이번 예제에서는 django의 model을 이용해서 db를 만들 것이다. 그 이유는 내가 django를 쓰기 때문에.... django 에서는 간편한 DB구성과 ORM , 쿼리문 들을 제공한다!!



In [ ]:
from __future__ import unicode_literals

from django.db import models

# Create your models here.
class result(models.Model):
    review=models.TextField(null=False,max_length=500)
    prediction=models.IntegerField()
    sentiment=models.IntegerField()

    def __str__(self):
        return self.review

In [ ]:
#django shell

from my_app.models import result

newresult=result()
result.review='this movie is really good'
result.prediction=1
result.sentiment=1

result.objects.all()

# View를 정의

get과 post 이라는 개념이 나오는데

사용자가 서버에 데이터를 보내는 방법이다

### get

https://www.google.co.kr get 방식으로 q(qestion) 인자로 django라는 값을 넘겨준다

https://www.google.co.kr/#q=django


### post

글쓰기 같은 매우 긴 텍스트나 값을 넘겨 줄때 사용

In [ ]:
from django.shortcuts import render
from django.views.generic.base import View
from forms import ReviewForm
# Create your views here.
from .movieclassfier.movie_model import classify,train
from .models import result

class review(View):
    
    def get(self,request):
        form=ReviewForm()
        return render(request,'review_form.html',{'form':form})
        
    def post(self,request):
        form=ReviewForm(request.POST)
        if form.is_valid():
            review=form.data['review']
            y,prob = classify(review)
            return render(request,'result.html',{'review': review,'y':y,'prob':prob})


이떄 뭔가 입력 받는 form 을 입력받기 위해 (html 에서의 < form> 태그 와 비슷?)

forms.py 에 RevieForm 을 정의 했다. 

django 에서 제공해주는 forms 를 사용하는 이유는 form validation 이 편하고 

modelform 이란걸 사용하여 모델이 잘 정의되 있다면 form을 쉽게 만들 수 있다.

In [ ]:
from django import forms

class ReviewForm(forms.Form):
    review = forms.CharField(
                            required=True,
                            widget=forms.Textarea,
                            error_messages={'required': 'review is required'}
                            )

# url patch 를 위해 urls.py 를 정의하자

정규 표현식을 사용하여 urls 을 각각의 view에 연결을 해준다 2가지 정도만 알고 있으면 문제가 없다

### ^ : 입력 문자열의 시작 부분에서 위치를 찾습니다

 ex)  	
 ^abc -> abcdef

 ^a?bc -> bcdef, abcdef

 ### $ :  입력 문자열의 끝 부분에서 위치를 찾습니다.  

ex)  
 t$ -> eat

 동$ -> 홍길동
 
 
 url(r'^admin/', admin.site.urls), : admin 을 포함하는 url을 admin.site.urls 로 연결  
 
 url(r'^$',views.review.as_view(), name='review'), : root url 을 review view로 연결  
 
 url(r'^feedback$',views.feedback.as_view(), name='feedback'), : 끝이 feedback 으로 끝나는 url 을 feedback view로 연결  

In [ ]:
"""python_ml_ch9 URL Configuration

The `urlpatterns` list routes URLs to views. For more information please see:
    https://docs.djangoproject.com/en/1.9/topics/http/urls/
Examples:
Function views
    1. Add an import:  from my_app import views
    2. Add a URL to urlpatterns:  url(r'^$', views.home, name='home')
Class-based views
    1. Add an import:  from other_app.views import Home
    2. Add a URL to urlpatterns:  url(r'^$', Home.as_view(), name='home')
Including another URLconf
    1. Add an import:  from blog import urls as blog_urls
    2. Import the include() function: from django.conf.urls import url, include
    3. Add a URL to urlpatterns:  url(r'^blog/', include(blog_urls))
"""
from django.conf.urls import url,include
from django.contrib import admin
from my_app import views
urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^$',views.review.as_view(), name='review'),
]

# 사용자에게 보여질 template 정의

# index.html 을 정의... 우리의 사이트에 쓰일 js 와 bootstrap 들을 받아온다.

다른 template 들은 이 index.html 을 확장해서 사용 할것이다.

왜 이렇게 하냐면...

불필요한 js, css 코드들이 중복되지 않고 html 파일들을 독립적으로 관리가능

In [ ]:
{% load staticfiles %}
<!DOCTYPE html>
<html lang="en">
<head>
  <title>Test</title>
  <meta charset="UTF-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1">

  <!-- bootstrap -->
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" integrity="sha384-1q8mTJOASx8j1Au+a5WDVnPi2lkFfwwEAa8hDDdjZlpLegxhjVME1fgjWPGmkzs7" crossorigin="anonymous">


</head>
<body>

  <div class="container">
    <div class="row">
  {% block content %}
  {% endblock %}
    </div>
  </div><!-- /container -->

  <script src="https://code.jquery.com/jquery-2.2.4.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js" integrity="sha384-0mSbJDEHialfmuBBQP6A4Qrprq5OVfW37PRR3j5ELqxss1yVqOtnepnHVP9aJ7xS" crossorigin="anonymous"></script>
</body>
</html>

# review_form.html 정의 

{% extends 'index.html' %}
{% load staticfiles %}
	
	{% block content %}
      <div class="col-md-12">
            <form class="form-horizontal" action="{% url 'review' %}" method="post">

              <div class="form-group">
                {% csrf_token %}
                <h1>Please enter your movie review</h1>
                
                {{form.review}}
                {{form.review.errors}}
                
              </div>
              <div class="form-group">
                <button type="submit" class="btn btn-primary">Submit review</button>

              </div>
            </form>
      </div>
    {% endblock %}

# Updating the movie review classifier

문장을 입력 받고 그것이 positive, negative 인지 classify 하는 부분 까지 구현이 되었다.

이제 하고 싶은 것은 우리의 예측값이 맞는지 틀린지 feedback 을 받는 것이다.

만약 'i love this movie' 를 positive 라 했고 그게 맞다고 사용자가 알려주면 그대로 positive 라고 db에 저장하고

'i love this movie'를 positive 라 했는데 사용자가 틀렸다고 feedback 해주면 negative 라고 저장 할 것이다.

그리고 이렇게 모인 데이터를 모아놨다가 다시 모델을 트레이닝 하는데 쓰고 싶다.

# feedback view 를 추가했다!!

In [ ]:
from django.shortcuts import render
from django.views.generic.base import View
from forms import ReviewForm
# Create your views here.
from .movieclassfier.movie_model import classify,train
from .models import result

class review(View):
    
    def get(self,request):
        form=ReviewForm()
        return render(request,'review_form.html',{'form':form})
        
    def post(self,request):
        form=ReviewForm(request.POST)
        if form.is_valid():
            review=form.data['review']
            y,prob = classify(review)
            return render(request,'result.html',{'review': review,'y':y,'prob':prob})

from django.views.decorators.csrf import csrf_exempt


class feedback(View):
    
    @csrf_exempt
    def post(self,request):
        feedback=request.POST['feedback_button']
        review=request.POST['review']
        prediction=request.POST['prediction']
        
        inv_label={'negative':0,'positive':1}
        y=inv_label[prediction]
        if feedback == 'Incorrcet':
            y=int(not(y))
        #train(review,y)
        
        result1=result()
        result1.review = review
        result1.prediction = int(inv_label[prediction])
        result1.sentiment = int(y)
        result1.save()
        
        return render(request,'thanks.html')
    

# urls.py 에도 feedback 과 연결

In [ ]:

from django.conf.urls import url,include
from django.contrib import admin
from my_app import views
urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^$',views.review.as_view(), name='review'),
    url(r'^feedback$',views.feedback.as_view(), name='feedback'),
]


# result.html 정의

prediction 결과와 그 확률값을 보여주고 

사용자에게 피드백을 form 형식으로 받음 

이때 귀찮아서 django form 안쓰고 그냥 html로 받았다.

In [ ]:
{% extends 'index.html' %}
{% load staticfiles %}
	
	{% block content %}
      <div class="col-md-12">
        <h1>your moview review:</h1>
        
        <p>{{ review }}</p>
        
        <h1>Prediction :</h1>
        
        <p>this mivew review is <strong>{{ y }}</strong> ( prob: {{ prob }} )</p>


          <form class="form-horizontal" action="{% url 'feedback' %}" method="post">
            {% csrf_token %}
            <div class="form-group">
              
              <input type=submit value='Correct' name='feedback_button'>
              <input type=submit value='Incorrect' name='feedback_button'>
              <input type=hidden value='{{ y }}' name='prediction'>
              <input type=hidden value='{{ review }}' name='review'>
              
            </div>
          </form>
            <a href={%url 'review'  %}><button type="submit" class="btn btn-primary">Submit another review</button></a>


    </div>

{% endblock %}
    


In [ ]:
{% extends 'index.html' %}
{% load staticfiles %}
	
	{% block content %}
      <h1>thanks your feedback</h1>
    {% endblock %}

# update.py 정의

모델불러오고 

sqlite3 db 에서 데이터를 가져온뒤에

partial training 시키는 예제

In [ ]:
import pickle
import sqlite3
import numpy as np
import os

# import HashingVectorizer from local dir
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):

    conn = sqlite3.connect(db_path)
    
    c = conn.cursor()
    c.execute('SELECT * from my_app_result')

    results = c.fetchmany(batch_size)
    while results:
        data = np.array(results)
        X = data[:, 1]
        y = data[:, 3].astype(int)

        classes = np.array([0, 1])
        X_train = vect.transform(X)
        model.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)

    conn.close()
    return model

cur_dir = os.path.dirname(os.path.abspath(__file__))

clf = pickle.load(open(os.path.join(cur_dir,
                  'pkl_objects',
                  'classifier.pkl'), 'rb'))
db = os.path.join(os.path.dirname(os.path.dirname(cur_dir)),'db.sqlite3')
print(db)


clf = update_model(db_path=db, model=clf, batch_size=10000)

# Uncomment the following lines if you are sure that
# you want to update your classifier.pkl file
# permanently.

pickle.dump(clf, open(os.path.join(cur_dir,
             'pkl_objects', 'classifier.pkl'), 'wb')
             , protocol=2)